In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    enableHiveSupport(). \
    appName('Basic Transformations'). \
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### importing functions

In [147]:
from pyspark.sql.functions import col, lpad, concat,date_format,to_date,\
count,lit,countDistinct,coalesce,expr,sum,avg,round,min, max,month,year,when,upper

# TASK
1. Get total number of flights as well as number of flights which are delayed in departure and number of flights delayed in arrival.
     - Output should contain 3 columns - **FlightCount, DepDelayedCount, ArrDelayedCount**
2. Get number of flights which are delayed in departure and number of flights delayed in arrival for each day along with number of flights departed for each day.
    - Output should contain 4 columns - **FlightDate, FlightCount, DepDelayedCount, ArrDelayedCount**
    - FlightDate should be of **yyyy-MM-dd format**.
    - Data should be **sorted** in ascending order by **flightDate**
3. Get all the flights which are departed late but arrived early (**IsArrDelayed is NO**).
    - Output should contain - **FlightCRSDepTime, UniqueCarrier, FlightNum, Origin, Dest, DepDelay, ArrDelay**
    - **FlightCRSDepTime** need to be computed using **Year, Month, DayOfMonth, CRSDepTime**
    - **FlightCRSDepTime** should be displayed using **yyyy-MM-dd HH:mm format**.
    - Output should be **sorted** by **FlightCRSDepTime** and then by the difference between **DepDelay** and **ArrDelay**
    - Also get the **count** of such flights


 >Building the Data Frame to explore basic transformations. Creating data frame using air traffic data.

In [4]:
!hdfs dfs -ls /workarea/airlines-part

Found 4 items
-rw-rw-rw-   2 hadoop supergroup          0 2023-04-08 14:31 /workarea/airlines-part/_SUCCESS
drwxrwxrwx   - hadoop supergroup          0 2023-04-08 14:31 /workarea/airlines-part/flightmonth=198710
drwxrwxrwx   - hadoop supergroup          0 2023-04-08 14:31 /workarea/airlines-part/flightmonth=198711
drwxrwxrwx   - hadoop supergroup          0 2023-04-08 14:31 /workarea/airlines-part/flightmonth=198712


In [5]:
airtraffic_path = "/workarea/airlines-part/flightmonth=198710"

In [6]:
airtraffic = spark. \
    read. \
    parquet(airtraffic_path)

In [7]:
airtraffic.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [8]:
airtraffic.select('Year', 'Month', 'DayOfMonth').distinct().show(5)

+----+-----+----------+
|Year|Month|DayOfMonth|
+----+-----+----------+
|1987|   10|         3|
|1987|   10|         6|
|1987|   10|        18|
|1987|   10|         4|
|1987|   10|        26|
+----+-----+----------+
only showing top 5 rows



In [9]:
airtraffic.select('Year', 'Month', 'DayOfMonth').distinct().count()

31

In [10]:
airtraffic.count()

448558

In [11]:
airtraffic.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|1987|   10|    

Getting  count of cancelled flights using **filter**

In [12]:
airtraffic.select('Cancelled').distinct().show()

+---------+
|Cancelled|
+---------+
|        1|
|        0|
+---------+



In [13]:
airtraffic.filter('Cancelled = 1').count()

2998

In [14]:
#Alternative format
airtraffic.filter(airtraffic['Cancelled'] == 1).count()

2998

In [15]:
#Alternative format
airtraffic.filter(airtraffic.Cancelled == 1).count()

2998

Get number of flights scheduled for departure from SFO airport.

In [16]:
airtraffic.filter("Origin = 'SFO'").count()

12195

In [17]:
airtraffic.filter(airtraffic['Origin'] == 'SFO').count()

12195

In [18]:
airtraffic.filter(airtraffic.Origin == 'SFO').count()

12195

Get number of flights that are departed from airport with out any delay.

In [19]:
airtraffic.select('IsDepDelayed').distinct().show()

+------------+
|IsDepDelayed|
+------------+
|         YES|
|          NO|
+------------+



In [20]:
airtraffic.filter("IsDepDelayed=='NO'").count()

270019

In [21]:
airtraffic.select('Year', 'Month', 'DayOfMonth').show(5)

+----+-----+----------+
|Year|Month|DayOfMonth|
+----+-----+----------+
|1987|   10|        27|
|1987|   10|         8|
|1987|   10|        30|
|1987|   10|        14|
|1987|   10|        23|
+----+-----+----------+
only showing top 5 rows



In [22]:
airtraffic.select(
    concat(
        col("Year"),
        lpad(col("Month"), 2, "0"),
        lpad(col("DayOfMonth"), 2, "0")
    ).alias('FlightDate')
).show(5)

+----------+
|FlightDate|
+----------+
|  19871027|
|  19871008|
|  19871030|
|  19871014|
|  19871023|
+----------+
only showing top 5 rows



In [23]:
airtraffic. \
    select(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    selectExpr("to_date(FlightDate, 'yyyyMMdd') AS FlightDate"). \
    show(5)

+----------+
|FlightDate|
+----------+
|1987-10-27|
|1987-10-08|
|1987-10-30|
|1987-10-14|
|1987-10-23|
+----------+
only showing top 5 rows



In [24]:
airtraffic. \
    select(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    selectExpr("date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') AS FlightDay"). \
    show(5)

+---------+
|FlightDay|
+---------+
|  Tuesday|
| Thursday|
|   Friday|
|Wednesday|
|   Friday|
+---------+
only showing top 5 rows



In [25]:
airtraffic. \
    select(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    select(date_format(to_date('FlightDate', 'yyyyMMdd'), 'EEEE').alias('FlightDay')). \
    show()

+---------+
|FlightDay|
+---------+
|  Tuesday|
| Thursday|
|   Friday|
|Wednesday|
|   Friday|
|   Friday|
|   Friday|
|  Tuesday|
|   Friday|
| Saturday|
|   Friday|
|Wednesday|
|   Friday|
| Thursday|
| Saturday|
| Saturday|
|   Sunday|
|   Sunday|
| Saturday|
|   Sunday|
+---------+
only showing top 20 rows



In [26]:
airtraffic.withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ).filter("""
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') = 'Sunday'
           """).count()

56120

In [27]:
airtraffic.withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ).filter("""IsDepDelayed = 'YES' AND 
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') = 'Sunday'
           """).count()

23552

In [28]:
#API Style
airtraffic.withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ).filter((col("IsDepDelayed") == "YES") &
           (date_format(to_date("FlightDate", "yyyyMMdd"), "EEEE") == "Sunday")
          ).count()

23552

#### Boolean Operators filtering data in dataframes
- Validating against multiple columns then we need to use boolean operations such as AND or OR or both.

>## Task1: 
Get count of flights which are departed late at origin and reach destination early or on time.

In [29]:
#airtraffic.columns

In [30]:
#airtraffic.select('ArrDelay').distinct().show()

In [31]:
airtraffic.filter((col("IsDepDelayed") == "YES") & (col("IsArrDelayed")=='NO')\
                  & (col("Cancelled")==0)).count()

33488

In [32]:
airtraffic. \
    filter("IsDepDelayed = 'YES' AND IsArrDelayed = 'NO' AND Cancelled = 0").count()

33488

>## Task2: 
Get count of flights which are departed early or on time but arrive late by at least 15 minutes.

In [33]:
airtraffic.filter((col("IsDepDelayed") == 'NO') & (col("ArrDelay") >=15)).count()

17472

>## Task3: 
Get number of flights which are departed late on Saturdays as well as on Sundays.

In [34]:
airtraffic.withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ).filter("""IsDepDelayed = 'YES' AND Cancelled = 0  AND 
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') = 'Saturday' OR
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') = 'Sunday'
           """).count()

78404

In [35]:
airtraffic.withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ).filter((col("IsDepDelayed") == "YES") & (col("Cancelled") == 0) &
           (date_format(to_date("FlightDate", "yyyyMMdd"), "EEEE") == 'Satuday') | \
            (date_format(to_date("FlightDate", "yyyyMMdd"), "EEEE") == 'Sunday')
          ).count()

56120

## Using **IN** Operator or **isin** Function

>## Task1:
    Get count of flights departed from following major airports - ORD, DFW, ATL, LAX, SFO.

In [36]:
#airtraffic.select("origin").distinct().show()

In [37]:
airtraffic. \
    filter("Origin IN ('ORD', 'DFW', 'ATL', 'LAX', 'SFO')"). \
    count()

90651

In [38]:
#c = col('x')
#help(c.isin)

In [39]:
airtraffic. \
    filter(col("Origin").isin("ORD", "DFW", "ATL", "LAX", "SFO")). \
    count()

90651

In [40]:
airtraffic.select(col("Diverted").isin([0,1])).count()

448558

In [41]:
airtraffic[airtraffic.Diverted.isin([0])].count()

447719

In [42]:
airtraffic[airtraffic.Diverted.isin([1])].count()

839

In [43]:
airtraffic. \
    filter(col("Origin").isin("ORD", "DFW", "ATL", "LAX", "SFO")). \
    count()

90651

>## Task2:
Get number of flights departed late on Sundays as well as on Saturdays. We can solve such kind of problems using IN operator as well.

In [44]:
airtraffic. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           IsDepDelayed = 'YES' AND Cancelled = 0 AND
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') IN
               ('Saturday', 'Sunday')
           """). \
    count()

45514

In [45]:
airtraffic. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") & (col("Cancelled") == 0) &
           (date_format(
               to_date("FlightDate", "yyyyMMdd"), "EEEE"
           ).isin("Saturday", "Sunday"))
          ). \
    count()

45514

## Using LIKE Operator or like Function
- **like** is primarily used for partial comparison (e.g.: Search for names which starts with Sco).
- We can use **like** to get results which starts with a pattern or ends with a pattern or contain the pattern.
- We can also use negation with like.
- Spark also provides **rlike** to take care of partial comparison using regular expression.


In [46]:
employees = [(1, "Scott", "Tiger", 1000.0, 10,
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, None,
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, '',
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 10,
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, bonus STRING, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )
employeesDF.show()


+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [47]:
employeesDF.filter("first_name LIKE 'Sco%'").show()

+-----------+----------+---------+------+-----+-------------+---------------+-----------+
|employee_id|first_name|last_name|salary|bonus|  nationality|   phone_number|        ssn|
+-----------+----------+---------+------+-----+-------------+---------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10|united states|+1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+-----+-------------+---------------+-----------+



In [48]:
employeesDF.filter(col('first_name').like('Sco%')).show()

+-----------+----------+---------+------+-----+-------------+---------------+-----------+
|employee_id|first_name|last_name|salary|bonus|  nationality|   phone_number|        ssn|
+-----------+----------+---------+------+-----+-------------+---------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10|united states|+1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+-----+-------------+---------------+-----------+



### Using BETWEEN Operator

>## Task1:
    Get count of flights departed late between 1987 October 1st to October 9th using FlightDate.

In [49]:
airtraffic.select("Year","Month","DayofMonth","DayOfWeek","DepTime","UniqueCarrier","FlightNum").show()

+----+-----+----------+---------+-------+-------------+---------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|UniqueCarrier|FlightNum|
+----+-----+----------+---------+-------+-------------+---------+
|1987|   10|        27|        2|    916|           PS|     1451|
|1987|   10|         8|        4|   1310|           PS|     1479|
|1987|   10|        30|        5|   1931|           PS|     1480|
|1987|   10|        14|        3|    824|           PS|     1489|
|1987|   10|        23|        5|   1807|           PS|     1500|
|1987|   10|         9|        5|   1415|           PS|     1509|
|1987|   10|        23|        5|    855|           PS|     1517|
|1987|   10|         6|        2|   1802|           PS|     1526|
|1987|   10|        23|        5|    827|           PS|     1548|
|1987|   10|         3|        6|   1945|           PS|     1564|
|1987|   10|        30|        5|   1853|           PS|     1570|
|1987|   10|         7|        3|    101|           PS|     1578|
|1987|   1

In [50]:
airtraffic. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           IsDepDelayed = 'YES' AND 
           Cancelled = 0 AND
           FlightDate BETWEEN 19871001 AND 19871009
          """). \
    count()

50675

In [51]:
#API format
airtraffic. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") & 
           (col("Cancelled") == 0) &
           (col("FlightDate").between(19871001, 19871009))
          ). \
    count()

50675

In [52]:
airtraffic. \
    filter((col("ArrDelay").between(15, 60))
          ). \
    count()

78051

### Dealing with Nulls while Filtering

In [53]:
employeesDF. \
    filter("bonus IS NOT NULL"). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [54]:
employeesDF. \
    filter("bonus <> ''"). \
    show()

+-----------+----------+---------+------+-----+-------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|  nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+-------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10|united states| +1 123 456 7890|123 45 6789|
|          4|      Bill|    Gomes|1500.0|   10|    AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+-------------+----------------+-----------+



In [55]:
employeesDF. \
    filter("bonus IS NOT NULL AND bonus <> ''"). \
    show()

+-----------+----------+---------+------+-----+-------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|  nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+-------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10|united states| +1 123 456 7890|123 45 6789|
|          4|      Bill|    Gomes|1500.0|   10|    AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+-------------+----------------+-----------+



In [56]:
employeesDF. \
    filter((col('bonus').isNotNull()) &
           (col('bonus') != '')
          ). \
    show()

+-----------+----------+---------+------+-----+-------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|  nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+-------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10|united states| +1 123 456 7890|123 45 6789|
|          4|      Bill|    Gomes|1500.0|   10|    AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+-------------+----------------+-----------+



In [57]:
employeesDF. \
    filter((~ col('bonus').isNull()) &
           (col('bonus') != '')
          ). \
    show()

+-----------+----------+---------+------+-----+-------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|  nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+-------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10|united states| +1 123 456 7890|123 45 6789|
|          4|      Bill|    Gomes|1500.0|   10|    AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+-------------+----------------+-----------+



In [58]:
employeesDF. \
    filter("bonus IS NULL"). \
    show()

+-----------+----------+---------+------+-----+-----------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+-----------+----------------+-----------+
|          2|     Henry|     Ford|1250.0| null|      India|+91 234 567 8901|456 78 9123|
+-----------+----------+---------+------+-----+-----------+----------------+-----------+



## Total Aggregations
- We can perform total aggregations directly on Dataframe or we can perform aggregations after grouping by a key(s).
- Here are the functions which we typically use to perform aggregations.
    - count
    - sum, avg
    - min, max

>## Task1:Aggregations
Get number of flights in the month of 1987 January.

In [59]:
airtraffic.count()

448558

In [60]:
airtraffic.select(count(lit(1)).alias('count')).show()

+------+
| count|
+------+
|448558|
+------+



In [61]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    describe(). \
    show()

+-------+------+------+------------------+
|summary|  Year| Month|        DayOfMonth|
+-------+------+------+------------------+
|  count|448558|448558|            448558|
|   mean|1987.0|  10.0|15.972980082843245|
| stddev|   0.0|   0.0|  8.93199415439287|
|    min|  1987|    10|                 1|
|    max|  1987|    10|                31|
+-------+------+------+------------------+



In [62]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    summary(). \
    show()

+-------+------+------+------------------+
|summary|  Year| Month|        DayOfMonth|
+-------+------+------+------------------+
|  count|448558|448558|            448558|
|   mean|1987.0|  10.0|15.972980082843245|
| stddev|   0.0|   0.0|  8.93199415439287|
|    min|  1987|    10|                 1|
|    25%|  1987|    10|                 8|
|    50%|  1987|    10|                16|
|    75%|  1987|    10|                24|
|    max|  1987|    10|                31|
+-------+------+------+------------------+



>## Task2:Aggregations
Get number of distinct dates from airtraffic data frame which is created using 1987 OCT data.

In [63]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    distinct(). \
    count()

31

In [64]:
airtraffic. \
    select(countDistinct('Year', 'Month', 'DayOfMonth').alias('countDistinct')). \
    show()

+-------------+
|countDistinct|
+-------------+
|           31|
+-------------+



In [65]:
employeesDF.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- bonus: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- ssn: string (nullable = true)



In [66]:
employeesDF. \
    select((coalesce(col('bonus').cast('int'), lit(0)) * col('salary') / lit(100)).alias('total_bonus')). \
    show()

+-----------+
|total_bonus|
+-----------+
|      100.0|
|        0.0|
|        0.0|
|      150.0|
+-----------+



In [67]:
employeesDF.select(((sum(coalesce(col('bonus').cast('int'),\
            lit(0)) * col('salary'))) / lit(100)).alias('total_bonus')). \
    show()

+-----------+
|total_bonus|
+-----------+
|      250.0|
+-----------+



>## Task3:Aggregations
    Get revenue generated for a given order from order_items.

In [68]:
order_items = spark.read.json('/data/retail_db_json/order_items')

In [69]:
order_id = input('Enter order_id:')

Enter order_id: 22


In [70]:
order_items. \
    filter(f'order_item_order_id = {int(order_id)}'). \
    show()

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+



In [71]:
order_items. \
    filter(f'order_item_order_id = {int(order_id)}'). \
    select(sum('order_item_subtotal').alias('order_revenue')). \
    show()

+-------------+
|order_revenue|
+-------------+
|         null|
+-------------+



#### Aggregate data using groupBy

- APIs
    1. groupBy
    2. rollup
    3. cube  
- Functions:
    - count
    - sum, avg
    - min, max



In [72]:
airtraffic.count()

448558

>## Task1: Aggregations groupBy
    Get number of flights scheduled each day for the month of OCT 1987

In [73]:
airtraffic. \
    groupBy(concat("year",
                   lpad("Month", 2, "0"),
                   lpad("DayOfMonth", 2, "0")
                  ).alias("FlightDate")
           ). \
    count(). \
    show()

+----------+-----+
|FlightDate|count|
+----------+-----+
|  19871001|14747|
|  19871030|14769|
|  19871002|14748|
|  19871018|14050|
|  19871021|14703|
|  19871029|14795|
|  19871020|14744|
|  19871019|14742|
|  19871014|14806|
|  19871006|14901|
|  19871003|13537|
|  19871013|14865|
|  19871010|13421|
|  19871022|14716|
|  19871012|14795|
|  19871004|14111|
|  19871011|14020|
|  19871025|13939|
|  19871028|14756|
|  19871007|14805|
+----------+-----+
only showing top 20 rows



In [74]:
airtraffic. \
    groupBy(concat("year",
                   lpad("Month", 2, "0"),
                   lpad("DayOfMonth", 2, "0")
                  ).alias("FlightDate")
           ). \
    agg(count(lit(1)).alias("FlightCount")). \
    show()

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|  19871001|      14747|
|  19871030|      14769|
|  19871002|      14748|
|  19871018|      14050|
|  19871021|      14703|
|  19871029|      14795|
|  19871020|      14744|
|  19871019|      14742|
|  19871014|      14806|
|  19871006|      14901|
|  19871003|      13537|
|  19871013|      14865|
|  19871010|      13421|
|  19871022|      14716|
|  19871012|      14795|
|  19871004|      14111|
|  19871011|      14020|
|  19871025|      13939|
|  19871028|      14756|
|  19871007|      14805|
+----------+-----------+
only showing top 20 rows



>## Task2: Aggregations groupBy
Get count of flights departed, total departure delay and average departure delay for each day over the month of OCT 1987

In [75]:
airtraffic. \
    filter('Cancelled = 0'). \
    groupBy(concat("year",
                   lpad("Month", 2, "0"),
                   lpad("DayOfMonth", 2, "0")
                  ).alias("FlightDate")
           ). \
    agg(
        count(lit(1)).alias("FlightCount"),
        sum('DepDelay').alias('TotalDepDelay'),
        round(avg('DepDelay'),2).alias('AverageDepDelay')
    ). \
    show(31)

+----------+-----------+-------------+---------------+
|FlightDate|FlightCount|TotalDepDelay|AverageDepDelay|
+----------+-----------+-------------+---------------+
|  19871001|      14594|      83842.0|           5.74|
|  19871030|      14699|      63726.0|           4.34|
|  19871002|      14672|      64590.0|            4.4|
|  19871018|      13997|      85456.0|           6.11|
|  19871021|      14613|      65676.0|           4.49|
|  19871029|      14717|      84895.0|           5.77|
|  19871020|      14612|      76144.0|           5.21|
|  19871019|      14656|      74703.0|            5.1|
|  19871014|      14739|      66810.0|           4.53|
|  19871006|      14807|      60301.0|           4.07|
|  19871003|      13447|      51141.0|            3.8|
|  19871013|      14756|      68141.0|           4.62|
|  19871010|      13354|      49686.0|           3.72|
|  19871022|      14634|      97431.0|           6.66|
|  19871012|      14431|      71423.0|           4.95|
|  1987100

>## Task3: Aggregations groupBy
Using order_items, get revenue for each order.

In [76]:
order_items.printSchema()

root
 |-- order_item_id: long (nullable = true)
 |-- order_item_order_id: long (nullable = true)
 |-- order_item_product_id: long (nullable = true)
 |-- order_item_product_price: double (nullable = true)
 |-- order_item_quantity: long (nullable = true)
 |-- order_item_subtotal: double (nullable = true)



In [77]:
order_items. \
    groupBy('order_item_order_id').\
        agg(
        round(sum('order_item_subtotal'),2).alias('order subtotal'), \
        round(avg('order_item_subtotal'),2).alias('revenue_per_order')).\
    show()

+-------------------+--------------+-----------------+
|order_item_order_id|order subtotal|revenue_per_order|
+-------------------+--------------+-----------------+
|              39713|        599.97|           299.99|
|              40395|        939.94|           187.99|
|              40436|        229.98|           114.99|
|              40557|        549.95|           137.49|
|              40634|       1119.88|           223.98|
|              41424|        829.95|           207.49|
|              41895|         649.9|           129.98|
|              41988|         669.9|            223.3|
|              42126|        899.88|           299.96|
|              42852|       1039.88|           207.98|
|              42969|        561.96|           187.32|
|              43367|       1079.87|           269.97|
|              44134|        759.92|           253.31|
|              44342|        119.98|           119.98|
|              44901|       1129.87|           225.97|
|         

>## Task4: Aggregations groupBy
Get min and max order_item_subtotal for each order id.

In [78]:
order_items. \
    groupBy('order_item_order_id').\
        agg(
        min('order_item_subtotal').alias('Min Value'), \
        max('order_item_subtotal').alias('Max Value')).\
    show()

+-------------------+---------+---------+
|order_item_order_id|Min Value|Max Value|
+-------------------+---------+---------+
|              39713|   199.99|   399.98|
|              40395|     50.0|   399.98|
|              40436|    99.99|   129.99|
|              40557|     50.0|    200.0|
|              40634|    99.99|   499.95|
|              41424|   129.99|   299.98|
|              41895|     50.0|   199.99|
|              41988|   129.99|   299.95|
|              42126|    249.9|   399.98|
|              42852|   129.99|   399.98|
|              42969|    31.99|   399.98|
|              43367|   129.99|   399.98|
|              44134|   119.98|   399.98|
|              44342|   119.98|   119.98|
|              44901|   129.99|   399.98|
|              45166|     50.0|   129.99|
|              45298|    79.98|   199.99|
|              45726|   119.97|   399.98|
|              46044|    39.98|   399.98|
|              46424|   109.94|   149.94|
+-------------------+---------+---

#### Aggregate data using **rollup**

In [79]:
orders = spark.read.json('/data/retail_db_json/orders')

In [80]:
orders.printSchema()

root
 |-- order_customer_id: long (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- order_status: string (nullable = true)



In [81]:
orders.show(5)

+-----------------+--------------------+--------+---------------+
|order_customer_id|          order_date|order_id|   order_status|
+-----------------+--------------------+--------+---------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|            12111|2013-07-25 00:00:...|       3|       COMPLETE|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|
|            11318|2013-07-25 00:00:...|       5|       COMPLETE|
+-----------------+--------------------+--------+---------------+
only showing top 5 rows



In [82]:
orders. \
    groupBy('order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    count()

364

In [83]:
orders.rollup?

Signature: orders.rollup(*cols: 'ColumnOrName') -> 'GroupedData'
Docstring:
Create a multi-dimensional rollup for the current :class:`DataFrame` using
the specified columns, so we can run aggregation on them.

.. versionadded:: 1.4.0

Examples
--------
>>> df.rollup("name", df.age).count().orderBy("name", "age").show()
+-----+----+-----+
| name| age|count|
+-----+----+-----+
| null|null|    2|
|Alice|null|    1|
|Alice|   2|    1|
|  Bob|null|    1|
|  Bob|   5|    1|
+-----+----+-----+
File:      /usr/local/spark/python/pyspark/sql/dataframe.py
Type:      method

In [84]:
orders. \
    rollup('order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_date'). \
    show(truncate=False)

+---------------------+-----------+
|order_date           |order_count|
+---------------------+-----------+
|null                 |68883      |
|2013-07-25 00:00:00.0|143        |
|2013-07-26 00:00:00.0|269        |
|2013-07-27 00:00:00.0|202        |
|2013-07-28 00:00:00.0|187        |
|2013-07-29 00:00:00.0|253        |
|2013-07-30 00:00:00.0|227        |
|2013-07-31 00:00:00.0|252        |
|2013-08-01 00:00:00.0|246        |
|2013-08-02 00:00:00.0|224        |
|2013-08-03 00:00:00.0|183        |
|2013-08-04 00:00:00.0|187        |
|2013-08-05 00:00:00.0|153        |
|2013-08-06 00:00:00.0|258        |
|2013-08-07 00:00:00.0|203        |
|2013-08-08 00:00:00.0|154        |
|2013-08-09 00:00:00.0|125        |
|2013-08-10 00:00:00.0|270        |
|2013-08-11 00:00:00.0|154        |
|2013-08-12 00:00:00.0|255        |
+---------------------+-----------+
only showing top 20 rows



In [85]:
orders. \
    rollup('order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_date'). \
    count()

365

>## Task1: Aggregations rollup
Get count of orders rolled up by month as well as date. You will see an additional record per month.

In [86]:
orders. \
    groupBy(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    show()

+-----------+--------------------+-----------+
|order_month|          order_date|order_count|
+-----------+--------------------+-----------+
|     201307|2013-07-25 00:00:...|        143|
|     201307|2013-07-26 00:00:...|        269|
|     201307|2013-07-27 00:00:...|        202|
|     201307|2013-07-28 00:00:...|        187|
|     201307|2013-07-29 00:00:...|        253|
|     201307|2013-07-30 00:00:...|        227|
|     201307|2013-07-31 00:00:...|        252|
|     201308|2013-08-01 00:00:...|        246|
|     201308|2013-08-02 00:00:...|        224|
|     201308|2013-08-03 00:00:...|        183|
|     201308|2013-08-04 00:00:...|        187|
|     201308|2013-08-05 00:00:...|        153|
|     201308|2013-08-06 00:00:...|        258|
|     201308|2013-08-07 00:00:...|        203|
|     201308|2013-08-08 00:00:...|        154|
|     201308|2013-08-09 00:00:...|        125|
|     201308|2013-08-10 00:00:...|        270|
|     201308|2013-08-11 00:00:...|        154|
|     201308|

In [87]:
orders. \
    groupBy(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    count()

364

In [88]:
orders. \
    rollup(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    show(truncate=False)

+-----------+---------------------+-----------+
|order_month|order_date           |order_count|
+-----------+---------------------+-----------+
|null       |null                 |68883      |
|201307     |null                 |1533       |
|201307     |2013-07-25 00:00:00.0|143        |
|201307     |2013-07-26 00:00:00.0|269        |
|201307     |2013-07-27 00:00:00.0|202        |
|201307     |2013-07-28 00:00:00.0|187        |
|201307     |2013-07-29 00:00:00.0|253        |
|201307     |2013-07-30 00:00:00.0|227        |
|201307     |2013-07-31 00:00:00.0|252        |
|201308     |null                 |5680       |
|201308     |2013-08-01 00:00:00.0|246        |
|201308     |2013-08-02 00:00:00.0|224        |
|201308     |2013-08-03 00:00:00.0|183        |
|201308     |2013-08-04 00:00:00.0|187        |
|201308     |2013-08-05 00:00:00.0|153        |
|201308     |2013-08-06 00:00:00.0|258        |
|201308     |2013-08-07 00:00:00.0|203        |
|201308     |2013-08-08 00:00:00.0|154  

In [89]:
orders. \
    rollup(month('order_date').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    show(truncate=False)

+-----------+---------------------+-----------+
|order_month|order_date           |order_count|
+-----------+---------------------+-----------+
|null       |null                 |68883      |
|1          |null                 |5908       |
|1          |2014-01-01 00:00:00.0|135        |
|1          |2014-01-02 00:00:00.0|111        |
|1          |2014-01-03 00:00:00.0|250        |
|1          |2014-01-04 00:00:00.0|129        |
|1          |2014-01-05 00:00:00.0|266        |
|1          |2014-01-06 00:00:00.0|155        |
|1          |2014-01-07 00:00:00.0|163        |
|1          |2014-01-08 00:00:00.0|122        |
|1          |2014-01-09 00:00:00.0|207        |
|1          |2014-01-10 00:00:00.0|241        |
|1          |2014-01-11 00:00:00.0|281        |
|1          |2014-01-12 00:00:00.0|215        |
|1          |2014-01-13 00:00:00.0|179        |
|1          |2014-01-14 00:00:00.0|209        |
|1          |2014-01-15 00:00:00.0|243        |
|1          |2014-01-16 00:00:00.0|194  

In [90]:
orders. \
    rollup(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    count()

378

>## Task2: Aggregations rollup
Get count of orders rolled up by year, month as well as date. You will see an additional record per month as well as per year.

In [91]:
orders.rollup(date_format('order_date', 'yyyy').alias('order_year'),date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    count()

380

In [92]:
orders.rollup(date_format('order_date', 'yyyy').alias('order_year'),date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_year', 'order_month', 'order_date'). \
    show()

+----------+-----------+--------------------+-----------+
|order_year|order_month|          order_date|order_count|
+----------+-----------+--------------------+-----------+
|      null|       null|                null|      68883|
|      2013|       null|                null|      30662|
|      2013|     201307|                null|       1533|
|      2013|     201307|2013-07-25 00:00:...|        143|
|      2013|     201307|2013-07-26 00:00:...|        269|
|      2013|     201307|2013-07-27 00:00:...|        202|
|      2013|     201307|2013-07-28 00:00:...|        187|
|      2013|     201307|2013-07-29 00:00:...|        253|
|      2013|     201307|2013-07-30 00:00:...|        227|
|      2013|     201307|2013-07-31 00:00:...|        252|
|      2013|     201308|                null|       5680|
|      2013|     201308|2013-08-01 00:00:...|        246|
|      2013|     201308|2013-08-02 00:00:...|        224|
|      2013|     201308|2013-08-03 00:00:...|        183|
|      2013|  

In [93]:
orders. \
    groupBy(
        year('order_date').alias('order_year'),
        date_format('order_date', 'yyyyMM').alias('order_month'), 
        'order_date'
    ). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_year', 'order_month', 'order_date'). \
    show()

+----------+-----------+--------------------+-----------+
|order_year|order_month|          order_date|order_count|
+----------+-----------+--------------------+-----------+
|      2013|     201307|2013-07-25 00:00:...|        143|
|      2013|     201307|2013-07-26 00:00:...|        269|
|      2013|     201307|2013-07-27 00:00:...|        202|
|      2013|     201307|2013-07-28 00:00:...|        187|
|      2013|     201307|2013-07-29 00:00:...|        253|
|      2013|     201307|2013-07-30 00:00:...|        227|
|      2013|     201307|2013-07-31 00:00:...|        252|
|      2013|     201308|2013-08-01 00:00:...|        246|
|      2013|     201308|2013-08-02 00:00:...|        224|
|      2013|     201308|2013-08-03 00:00:...|        183|
|      2013|     201308|2013-08-04 00:00:...|        187|
|      2013|     201308|2013-08-05 00:00:...|        153|
|      2013|     201308|2013-08-06 00:00:...|        258|
|      2013|     201308|2013-08-07 00:00:...|        203|
|      2013|  

In [94]:
orders. \
    rollup(
        year('order_date').alias('order_year'),
        date_format('order_date', 'yyyyMM').alias('order_month'), 
        'order_date'
    ). \
    agg(count(lit(1)).alias('order_count')). \
    filter("order_month = 201401"). \
    orderBy('order_year', 'order_month', 'order_date'). \
    show(32)

+----------+-----------+--------------------+-----------+
|order_year|order_month|          order_date|order_count|
+----------+-----------+--------------------+-----------+
|      2014|     201401|                null|       5908|
|      2014|     201401|2014-01-01 00:00:...|        135|
|      2014|     201401|2014-01-02 00:00:...|        111|
|      2014|     201401|2014-01-03 00:00:...|        250|
|      2014|     201401|2014-01-04 00:00:...|        129|
|      2014|     201401|2014-01-05 00:00:...|        266|
|      2014|     201401|2014-01-06 00:00:...|        155|
|      2014|     201401|2014-01-07 00:00:...|        163|
|      2014|     201401|2014-01-08 00:00:...|        122|
|      2014|     201401|2014-01-09 00:00:...|        207|
|      2014|     201401|2014-01-10 00:00:...|        241|
|      2014|     201401|2014-01-11 00:00:...|        281|
|      2014|     201401|2014-01-12 00:00:...|        215|
|      2014|     201401|2014-01-13 00:00:...|        179|
|      2014|  

### Aggregate data using cube

>## Task1: Aggregations cube
    Get count of orders by date using cube.

In [95]:
orders. \
    groupBy('order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_date'). \
    show(5)

+--------------------+-----------+
|          order_date|order_count|
+--------------------+-----------+
|2013-07-25 00:00:...|        143|
|2013-07-26 00:00:...|        269|
|2013-07-27 00:00:...|        202|
|2013-07-28 00:00:...|        187|
|2013-07-29 00:00:...|        253|
+--------------------+-----------+
only showing top 5 rows



In [96]:
orders. \
    groupBy('order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    count()

364

In [97]:
#orders.cube?

In [98]:
orders. \
    cube('order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_date'). \
    show()

+--------------------+-----------+
|          order_date|order_count|
+--------------------+-----------+
|                null|      68883|
|2013-07-25 00:00:...|        143|
|2013-07-26 00:00:...|        269|
|2013-07-27 00:00:...|        202|
|2013-07-28 00:00:...|        187|
|2013-07-29 00:00:...|        253|
|2013-07-30 00:00:...|        227|
|2013-07-31 00:00:...|        252|
|2013-08-01 00:00:...|        246|
|2013-08-02 00:00:...|        224|
|2013-08-03 00:00:...|        183|
|2013-08-04 00:00:...|        187|
|2013-08-05 00:00:...|        153|
|2013-08-06 00:00:...|        258|
|2013-08-07 00:00:...|        203|
|2013-08-08 00:00:...|        154|
|2013-08-09 00:00:...|        125|
|2013-08-10 00:00:...|        270|
|2013-08-11 00:00:...|        154|
|2013-08-12 00:00:...|        255|
+--------------------+-----------+
only showing top 20 rows



In [99]:
orders. \
    cube('order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_date'). \
    count()

365

>## Task2:Aggregattions cube
Get count of orders by month as well as date using cube. You will see additional records per month.

In [100]:
orders. \
    groupBy(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    show()

+-----------+--------------------+-----------+
|order_month|          order_date|order_count|
+-----------+--------------------+-----------+
|     201307|2013-07-25 00:00:...|        143|
|     201307|2013-07-26 00:00:...|        269|
|     201307|2013-07-27 00:00:...|        202|
|     201307|2013-07-28 00:00:...|        187|
|     201307|2013-07-29 00:00:...|        253|
|     201307|2013-07-30 00:00:...|        227|
|     201307|2013-07-31 00:00:...|        252|
|     201308|2013-08-01 00:00:...|        246|
|     201308|2013-08-02 00:00:...|        224|
|     201308|2013-08-03 00:00:...|        183|
|     201308|2013-08-04 00:00:...|        187|
|     201308|2013-08-05 00:00:...|        153|
|     201308|2013-08-06 00:00:...|        258|
|     201308|2013-08-07 00:00:...|        203|
|     201308|2013-08-08 00:00:...|        154|
|     201308|2013-08-09 00:00:...|        125|
|     201308|2013-08-10 00:00:...|        270|
|     201308|2013-08-11 00:00:...|        154|
|     201308|

In [101]:
orders. \
    groupBy(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    count()

364

In [102]:
orders. \
    rollup(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    show()

+-----------+--------------------+-----------+
|order_month|          order_date|order_count|
+-----------+--------------------+-----------+
|       null|                null|      68883|
|     201307|                null|       1533|
|     201307|2013-07-25 00:00:...|        143|
|     201307|2013-07-26 00:00:...|        269|
|     201307|2013-07-27 00:00:...|        202|
|     201307|2013-07-28 00:00:...|        187|
|     201307|2013-07-29 00:00:...|        253|
|     201307|2013-07-30 00:00:...|        227|
|     201307|2013-07-31 00:00:...|        252|
|     201308|                null|       5680|
|     201308|2013-08-01 00:00:...|        246|
|     201308|2013-08-02 00:00:...|        224|
|     201308|2013-08-03 00:00:...|        183|
|     201308|2013-08-04 00:00:...|        187|
|     201308|2013-08-05 00:00:...|        153|
|     201308|2013-08-06 00:00:...|        258|
|     201308|2013-08-07 00:00:...|        203|
|     201308|2013-08-08 00:00:...|        154|
|     201308|

In [103]:
orders. \
    rollup(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    count()

378

In [104]:
orders. \
    cube(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    show()

+-----------+--------------------+-----------+
|order_month|          order_date|order_count|
+-----------+--------------------+-----------+
|       null|                null|      68883|
|       null|2013-07-25 00:00:...|        143|
|       null|2013-07-26 00:00:...|        269|
|       null|2013-07-27 00:00:...|        202|
|       null|2013-07-28 00:00:...|        187|
|       null|2013-07-29 00:00:...|        253|
|       null|2013-07-30 00:00:...|        227|
|       null|2013-07-31 00:00:...|        252|
|       null|2013-08-01 00:00:...|        246|
|       null|2013-08-02 00:00:...|        224|
|       null|2013-08-03 00:00:...|        183|
|       null|2013-08-04 00:00:...|        187|
|       null|2013-08-05 00:00:...|        153|
|       null|2013-08-06 00:00:...|        258|
|       null|2013-08-07 00:00:...|        203|
|       null|2013-08-08 00:00:...|        154|
|       null|2013-08-09 00:00:...|        125|
|       null|2013-08-10 00:00:...|        270|
|       null|

In [105]:
orders. \
    cube(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    filter('order_month IS NOT NULL'). \
    orderBy('order_month', 'order_date'). \
    show()

+-----------+--------------------+-----------+
|order_month|          order_date|order_count|
+-----------+--------------------+-----------+
|     201307|                null|       1533|
|     201307|2013-07-25 00:00:...|        143|
|     201307|2013-07-26 00:00:...|        269|
|     201307|2013-07-27 00:00:...|        202|
|     201307|2013-07-28 00:00:...|        187|
|     201307|2013-07-29 00:00:...|        253|
|     201307|2013-07-30 00:00:...|        227|
|     201307|2013-07-31 00:00:...|        252|
|     201308|                null|       5680|
|     201308|2013-08-01 00:00:...|        246|
|     201308|2013-08-02 00:00:...|        224|
|     201308|2013-08-03 00:00:...|        183|
|     201308|2013-08-04 00:00:...|        187|
|     201308|2013-08-05 00:00:...|        153|
|     201308|2013-08-06 00:00:...|        258|
|     201308|2013-08-07 00:00:...|        203|
|     201308|2013-08-08 00:00:...|        154|
|     201308|2013-08-09 00:00:...|        125|
|     201308|

In [106]:
orders. \
    cube(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    count()

742

>## Task3: Aggregations cube
    Get count of orders by year, month as well as date using cube. You will see additional records per month as well as per year.

In [107]:
orders. \
    groupBy(
        year('order_date').alias('order_year'),
        date_format('order_date', 'yyyyMM').alias('order_month'), 
        'order_date'
    ). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_year', 'order_month', 'order_date'). \
    show()

+----------+-----------+--------------------+-----------+
|order_year|order_month|          order_date|order_count|
+----------+-----------+--------------------+-----------+
|      2013|     201307|2013-07-25 00:00:...|        143|
|      2013|     201307|2013-07-26 00:00:...|        269|
|      2013|     201307|2013-07-27 00:00:...|        202|
|      2013|     201307|2013-07-28 00:00:...|        187|
|      2013|     201307|2013-07-29 00:00:...|        253|
|      2013|     201307|2013-07-30 00:00:...|        227|
|      2013|     201307|2013-07-31 00:00:...|        252|
|      2013|     201308|2013-08-01 00:00:...|        246|
|      2013|     201308|2013-08-02 00:00:...|        224|
|      2013|     201308|2013-08-03 00:00:...|        183|
|      2013|     201308|2013-08-04 00:00:...|        187|
|      2013|     201308|2013-08-05 00:00:...|        153|
|      2013|     201308|2013-08-06 00:00:...|        258|
|      2013|     201308|2013-08-07 00:00:...|        203|
|      2013|  

In [108]:
orders. \
    groupBy(
        year('order_date').alias('order_year'),
        date_format('order_date', 'yyyyMM').alias('order_month'), 
        'order_date'
    ). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_year', 'order_month', 'order_date'). \
    count()

364

In [109]:
orders. \
    rollup(
        year('order_date').alias('order_year'),
        date_format('order_date', 'yyyyMM').alias('order_month'), 
        'order_date'
    ). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_year', 'order_month', 'order_date'). \
    show()

+----------+-----------+--------------------+-----------+
|order_year|order_month|          order_date|order_count|
+----------+-----------+--------------------+-----------+
|      null|       null|                null|      68883|
|      2013|       null|                null|      30662|
|      2013|     201307|                null|       1533|
|      2013|     201307|2013-07-25 00:00:...|        143|
|      2013|     201307|2013-07-26 00:00:...|        269|
|      2013|     201307|2013-07-27 00:00:...|        202|
|      2013|     201307|2013-07-28 00:00:...|        187|
|      2013|     201307|2013-07-29 00:00:...|        253|
|      2013|     201307|2013-07-30 00:00:...|        227|
|      2013|     201307|2013-07-31 00:00:...|        252|
|      2013|     201308|                null|       5680|
|      2013|     201308|2013-08-01 00:00:...|        246|
|      2013|     201308|2013-08-02 00:00:...|        224|
|      2013|     201308|2013-08-03 00:00:...|        183|
|      2013|  

In [110]:
orders. \
    rollup(
        year('order_date').alias('order_year'),
        date_format('order_date', 'yyyyMM').alias('order_month'), 
        'order_date'
    ). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_year', 'order_month', 'order_date'). \
    count()

380

In [111]:
orders. \
    cube(
        year('order_date').alias('order_year'),
        date_format('order_date', 'yyyyMM').alias('order_month'), 
        'order_date'
    ). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_year', 'order_month', 'order_date'). \
    count()

1485

In [112]:
orders. \
    cube(
        year('order_date').alias('order_year'),
        date_format('order_date', 'yyyyMM').alias('order_month'), 
        'order_date'
    ). \
    agg(count(lit(1)).alias('order_count')). \
    filter("order_date LIKE '2014-01%'"). \
    orderBy('order_year', 'order_month', 'order_date'). \
    show(150)

+----------+-----------+--------------------+-----------+
|order_year|order_month|          order_date|order_count|
+----------+-----------+--------------------+-----------+
|      null|       null|2014-01-01 00:00:...|        135|
|      null|       null|2014-01-02 00:00:...|        111|
|      null|       null|2014-01-03 00:00:...|        250|
|      null|       null|2014-01-04 00:00:...|        129|
|      null|       null|2014-01-05 00:00:...|        266|
|      null|       null|2014-01-06 00:00:...|        155|
|      null|       null|2014-01-07 00:00:...|        163|
|      null|       null|2014-01-08 00:00:...|        122|
|      null|       null|2014-01-09 00:00:...|        207|
|      null|       null|2014-01-10 00:00:...|        241|
|      null|       null|2014-01-11 00:00:...|        281|
|      null|       null|2014-01-12 00:00:...|        215|
|      null|       null|2014-01-13 00:00:...|        179|
|      null|       null|2014-01-14 00:00:...|        209|
|      null|  

In [114]:
orders. \
    cube(
        year('order_date').alias('order_year'),
        date_format('order_date', 'yyyyMM').alias('order_month'), 
        'order_date'
    ). \
    agg(count(lit(1)).alias('order_count')). \
    filter("order_date LIKE '2014-01%'"). \
    orderBy('order_year', 'order_month', 'order_date'). \
    count()

124

## Overview of Sorting Data Frames
- We can use orderBy or sort to sort the data.
- We can perform composite sorting by passing multiple columns or expressions.
- By default data is sorted in ascending order, we can change it to descending by applying desc() function on the column or expression.
- If the sort column contain null values those will come first. We can change the position of nulls to last.


>## Task1
Get daily count of cancelled flights where data is sorted in ascending order by count of cancelled flights.

In [115]:
airtraffic. \
    filter('cancelled = 1'). \
    groupBy(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    agg(count(lit(1)).alias('FlightCount')). \
    show()

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|  19871001|        153|
|  19871030|         70|
|  19871002|         76|
|  19871018|         53|
|  19871021|         90|
|  19871029|         78|
|  19871020|        132|
|  19871019|         86|
|  19871014|         67|
|  19871006|         94|
|  19871003|         90|
|  19871013|        109|
|  19871022|         82|
|  19871010|         67|
|  19871004|        111|
|  19871012|        364|
|  19871011|         45|
|  19871025|        113|
|  19871028|        174|
|  19871007|         39|
+----------+-----------+
only showing top 20 rows



In [117]:
airtraffic. \
    filter('cancelled = 1'). \
    groupBy(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    agg(count(lit(1)).alias('FlightCount')). \
    orderBy('FlightCount'). \
    show(5)

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|  19871007|         39|
|  19871011|         45|
|  19871015|         47|
|  19871008|         50|
|  19871018|         53|
+----------+-----------+
only showing top 5 rows



In [120]:
airtraffic. \
    filter('cancelled = 1'). \
    groupBy(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    agg(count(lit(1)).alias('FlightCount')). \
    orderBy(col('FlightCount').asc()). \
    show(5)

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|  19871007|         39|
|  19871011|         45|
|  19871015|         47|
|  19871008|         50|
|  19871018|         53|
+----------+-----------+
only showing top 5 rows



>## Task2:
Get daily count of cancelled flights where data is sorted in descending order by count of cancelled flights.

In [122]:
airtraffic. \
    filter('cancelled = 1'). \
    groupBy(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    agg(count(lit(1)).alias('FlightCount')). \
    orderBy(col('FlightCount').desc()). \
    show(5)

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|  19871012|        364|
|  19871028|        174|
|  19871001|        153|
|  19871005|        134|
|  19871020|        132|
+----------+-----------+
only showing top 5 rows



>## Task3
- Project Year, Month, DayOfMonth, CRSDepTime and Origin for this task.
- Sort the data based up on year, month, day of month and then using scheduled time. Data should be sorted in ascending order by year, month and day of month then in descending order by scheduled time.


In [125]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth', 'CRSDepTime', 'Origin'). \
    show(5)

+----+-----+----------+----------+------+
|Year|Month|DayOfMonth|CRSDepTime|Origin|
+----+-----+----------+----------+------+
|1987|   10|        27|       915|   SFO|
|1987|   10|         8|      1310|   SAN|
|1987|   10|        30|      1915|   SFO|
|1987|   10|        14|       815|   SNA|
|1987|   10|        23|      1745|   SMF|
+----+-----+----------+----------+------+
only showing top 5 rows



In [139]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth', 'CRSDepTime', 'Origin'). \
    orderBy('Year', 'Month', 'DayOfMonth', 'CRSDepTime'). \
    show(5)

+----+-----+----------+----------+------+
|Year|Month|DayOfMonth|CRSDepTime|Origin|
+----+-----+----------+----------+------+
|1987|   10|         1|         1|   SFO|
|1987|   10|         1|         5|   MCO|
|1987|   10|         1|         5|   ICT|
|1987|   10|         1|         5|   LAX|
|1987|   10|         1|         8|   LAS|
+----+-----+----------+----------+------+
only showing top 5 rows



In [136]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth', 'CRSDepTime', 'Origin'). \
    orderBy('Year', 'Month', 'DayOfMonth', col('CRSDepTime').desc()). \
    show(5)

+----+-----+----------+----------+------+
|Year|Month|DayOfMonth|CRSDepTime|Origin|
+----+-----+----------+----------+------+
|1987|   10|         1|      2359|   LAX|
|1987|   10|         1|      2359|   SEA|
|1987|   10|         1|      2359|   LAX|
|1987|   10|         1|      2359|   OKC|
|1987|   10|         1|      2359|   SFO|
+----+-----+----------+----------+------+
only showing top 5 rows



>## Task4:
Create employees Data Frame and get employees data in ascending order by nationality. However, data related to United States should come at top always.

In [142]:
employees = [(1, "Scott", "Tiger", 1000.0, 10,
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, None,
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, '',
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 2,
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]
mployeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, bonus STRING, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )
employeesDF.show()


+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [145]:
when?

Signature: when(condition: pyspark.sql.column.Column, value: Any) -> pyspark.sql.column.Column
Docstring:
Evaluates a list of conditions and returns one of multiple possible result expressions.
If :func:`pyspark.sql.Column.otherwise` is not invoked, None is returned for unmatched
conditions.

.. versionadded:: 1.4.0

Parameters
----------
condition : :class:`~pyspark.sql.Column`
    a boolean :class:`~pyspark.sql.Column` expression.
value :
    a literal value, or a :class:`~pyspark.sql.Column` expression.

Examples
--------
>>> df.select(when(df['age'] == 2, 3).otherwise(4).alias("age")).collect()
[Row(age=3), Row(age=4)]

>>> df.select(when(df.age == 2, df.age + 1).alias("age")).collect()
[Row(age=3), Row(age=None)]
File:      /usr/local/spark/python/pyspark/sql/functions.py
Type:      function

In [148]:
employeesDF. \
    withColumn('sort_column', when(upper(col('nationality')) == 'UNITED STATES', 0).otherwise(1)). \
    orderBy('sort_column', 'nationality'). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|sort_column|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|          0|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|          1|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|          1|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|          1|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+



In [149]:
employeesDF. \
    withColumn(
        'sort_column', 
        expr("""CASE WHEN upper(nationality) = 'UNITED STATES' THEN 0 else 1 END""")
    ). \
    orderBy('sort_column', 'nationality'). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|sort_column|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|          0|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|          1|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|          1|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|          1|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+



>## Task5
Sort the data in employeesDF using bonus. Data should be sorted numerically and null and empty values should come at the end.

In [150]:
employeesDF.show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [151]:
employeesDF. \
    orderBy('bonus'). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [152]:
employeesDF. \
    orderBy(col('bonus').cast('int')). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [154]:
#c = col('X')
#help(c)
#help(c.asc_nulls_last)


In [155]:
employeesDF. \
    orderBy(employeesDF.bonus.cast('int').asc_nulls_last()). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+

